Import Pyomo, course code, and Pandas.

In [20]:
from pyomo.environ import *
from brooks import *
import pandas as pd

Define data.  The index_col parameter gives the row indexes.  By default, the function assumes the first row contains column names.

In [21]:
school_data = pd.read_excel("schools.xlsx", 
                          sheet_name="Sheet1",
                          index_col=0)

col_names = list(school_data)
I = [i for i in col_names if "Input" in i]
J = [j for j in col_names if "Output" in j]
S = list(school_data.index)

input_data = school_data[I].stack().to_dict()
output_data = school_data[J].stack().to_dict()

Declare a model object.

In [22]:
model = Model()

Declare a parameter for the objective function coefficients.

In [23]:
if hasattr(model, 'obj_coef'):
    model.delete(model.obj_coef)
model.obj_coef = Param(J, mutable=True)

Declare a parameter for the normalization constraint coeffcients.

In [24]:
if hasattr(model, 'norm_coef'):
    model.delete(model.norm_coef)
model.norm_coef = Param(I, mutable=True)

Declare nonnegative variables for costs of inputs.

In [25]:
if hasattr(model, 'x'):
    model.delete(model.x)
model.x = Var(I, domain=NonNegativeReals)

Define nonnegative variables for prices of outputs.

In [26]:
if hasattr(model, 'y'):
    model.delete(model.y)
model.y = Var(J, domain=NonNegativeReals)

Define the objective function.

In [27]:
if hasattr(model, 'total_value_objective'):
    model.delete(model.total_value_objective)
model.total_value_objective = Objective(
    expr = sum(model.obj_coef[j]*model.y[j] for j in J), 
    sense=maximize)

Specify the efficiency limit constraint. 

In [28]:
if hasattr(model, 'efficiency_constraint'):
    model.delete(model.efficiency_constraint)
model.efficiency_constraint = ConstraintList()
for s in S:
    model.efficiency_constraint.add(sum(output_data[s,j]*model.y[j] for j in J) <= 
                              sum(input_data[s,i]*model.x[i] for i in I))

Specify the normalization constraint.

In [29]:
if hasattr(model, 'normalization_constraint'):
    model.delete(model.normalization_constraint)
model.normalization_constraint =  Constraint(
    expr=sum(model.norm_coef[i]*model.x[i] for i in I) == 1)
model.pprint()

5 Set Declarations
    efficiency_constraint_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {1, 2, 3, 4}
    norm_coef_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {'Input 1', 'Input 2', 'Input 3'}
    obj_coef_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {'Output 1', 'Output 2', 'Output 3'}
    x_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {'Input 1', 'Input 2', 'Input 3'}
    y_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {'Output 1', 'Output 2', 'Output 3'}

2 Param Declarations
    norm_coef : Size=3, Index=norm_coef_index, Domain=Any, Default=None, Mutabl

Specify solver and solve.


In [30]:
for s in S:
    model.obj_coef.store_values({j: output_data[s,j] for j in J})
    model.norm_coef.store_values({i: input_data[s,i] for i in I})
    solver = SolverFactory('glpk')
    status = solver.solve(model)
    print("%s objective %f" % (s, value(model.total_value_objective)))

Alton objective 1.000000
Beeks objective 1.000000
Carey objective 1.000000
Delancey objective 0.858209
